In [1]:
# import all the typical toolboxes...
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.interpolate import interp1d
from numpy import*
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cantera as ct

# T$_{AD}$ example:

Now let's look at the adiabatic flame temperature example -- first we'll consider a rich-burning ethane flame with a $\phi$ of 1.5:

$$ C_2H_6 + \frac{a_s}{\phi}\left(O_2 + 3.76N_2\right) \rightarrow Products\left(CO_2, H_2O, N_2, CO, H_2, NO, O_2\right) \text{ at } T_{ad} $$

In [2]:
# The easiest way to use pandas.read_csv is to have the *.csv file in the folder our jupyter notebook is in:
data = pd.read_csv("lnkp_v_temp.csv")
data #show the data

,T(K),CO+1/2O eq CO2,1/2H2 eq H,H2+1/2O eq H2O,1/2N2 eq N,1/2O2+1/2N2 eq NO,O2+1/2N2 eq NO2,1/2O2 eq O,1/2H2+1/2O2 eq OH
0,200.00,159.6811,-125.0847,139.9748,-277.2662,-53.3900,-27.9571,-142.7264,-20.5011
1,298.15,103.7531,-82.0006,92.2077,-183.7606,-35.3306,-21.1037,-93.4813,-13.1346
2,400.00,74.6618,-59.5794,67.3216,-135.1804,-25.9536,-17.6196,-67.8546,-9.3031
3,600.00,46.2378,-37.6127,42.8974,-87.6896,-16.7997,-14.2742,-42.7592,-5.5674
4,800.00,32.0310,-26.5668,30.5926,-63.8855,-12.2212,-12.6124,-30.1586,-3.7106
5,1000.00,23.5225,-19.9031,23.1622,-49.5719,-9.4731,-11.6103,-22.5729,-2.6070
6,1200.00,17.8648,-15.4375,18.1826,-40.0116,-7.6402,-10.9354,-17.5017,-1.8784
7,1400.00,13.8360,-12.2327,14.6115,-33.1715,-6.3305,-10.4473,-13.8705,-1.3634
8,1600.00,10.8245,-9.8185,11.9246,-28.0337,-5.3480,-10.0761,-11.1413,-0.9802
9,1800.00,8.4906,-7.9333,9.8304,-24.0323,-4.5838,-9.7835,-9.0144,-0.6848


In [3]:
# Now let's setup some Kp fxns:
TT = data['T(K)']
fkp_co2 = data['CO+1/2O eq CO2']
lnkp_co2 = interp1d(TT,fkp_co2) #setup a function that will interpolate within the bounds of our table above

fkp_h2o = data['H2+1/2O eq H2O']
lnkp_h2o = interp1d(TT,fkp_h2o)

# Note that we now also need a one more Kp equation ... 
# given what we've assumed in our products, it makes the most sense to select the one for NO
fkp_no = data['1/2O2+1/2N2 eq NO']
lnkp_no = interp1d(TT,fkp_no)

In [4]:
# Now we need to import our enthalpy coefficients...

# Here is an example with just air so we are familiar with how this will all work...
air_pcoef = [-1.966e-9, 0.4802e-5, 0.1967e-2, 28.11] #thermo coeff form Table A-2 in thermo book
pair = poly1d(air_pcoef)/28.97 #Cp_air [kJ/kg-K]
print("The calculated cp at 300K = %1.4f kJ/kgK" % pair(300)) #Cp_air at 300K
int_cpair = pair.integ() #creates the integrand of our Cp fxn for air
dh_air = int_cpair(400) - int_cpair(300) #evaluates the integral of CpdT from 300 to 400
print("For ex, the dh for air from 300K to 400K = %1.2f kJ/kg" % dh_air) #shows dh_air

The calculated cp at 300K = 1.0038 kJ/kgK
For ex, the dh for air from 300K to 400K = 101.16 kJ/kg


#### CoolProp version...

Alternatively, we could have imported CoolProp to do this stuff for us. Though, note that the gasses of interest need to be listed here: http://www.coolprop.org/fluid_properties/PurePseudoPure.html#list-of-fluids

The rub with CoolProp is that it doesn't have heats (i.e., enthalpies) of formation, so it's really only of limited use...

In [5]:
# Alternatively, we can use CoolProp
import CoolProp.CoolProp as CP

R = 8.314 #ideal gas const
T = {}
T[1] = 300 #K
T[2] = 400 #K
P = 1e5 #Pa

# note dividing by 1000 b/c base units are J, kg, K, etc.
u_1 = CP.PropsSI('U','T',T[1],'P',P,'Air')/1000
h_1 = CP.PropsSI('H','T',T[1],'P',P,'Air')/1000
s_1 = CP.PropsSI('S','T',T[1],'P',P,'Air')/1000
cpair = CP.PropsSI('C','T',T[1],'P',P,'Air')/1000

print("the internal energy is %1.3f kJ/kg" % u_1)
print("the enthalpy is %1.3f kJ/kg" % h_1)
print("the entropy is %1.3f kJ/kgK" % s_1)
print('the specific heat capacity is %1.3f kJ/kgK' % cpair)

h_2 = CP.PropsSI('H','T',T[2],'P',P,'Air')/1000
dh = h_2-h_1
print("dh(300 to 400) = %1.3f kJ/kg" % dh)

the internal energy is 340.213 kJ/kg
the enthalpy is 426.301 kJ/kg
the entropy is 3.891 kJ/kgK
the specific heat capacity is 1.006 kJ/kgK
dh(300 to 400) = 100.955 kJ/kg


### Setting up our thermodynamic polynomials...

Before we can begin, we need to setup a bunch of thermodyanmic constants to deal with the changes in enthalpy in our problem. In other words, we need to set up our dh polynomials. Now, just to keep things simple, I'm going to use the ones from the Thermo book (mostly becuase they are only 3rd order polynomials). If you want to match what you'd get from GasEq or Cantera then you'll need to use the full ones from Turns...

In [6]:
## heats of formation in kJ/kmol-K
hfc2h6 = -84680
hfco2 = -393520
hfh2o = -241820
hfno = 90297
hfco = -110541

## make our dh_s functions ... note you could also try pulling these coeff from a table with pandas
co2_pcoef = [7.469e-9, -3.501e-5, 5.981e-2, 22.26] #thermo polynomial coeff for cp_co2
pco2 = poly1d(co2_pcoef) #Cp_co2 [kJ/kmol-K]
hsco2 = pco2.integ() #h_s function
#
h2o_pcoef = [-3.595e-9, 1.055e-5, 0.1923e-2, 32.24] #thermo polynomial coeff for cp_h2o
ph2o = poly1d(h2o_pcoef) #Cp_h2o [kJ/kmol-K]
hsh2o = ph2o.integ() #h_s function
#
n2_pcoef = [-2.873e-9, 0.8081e-5, -0.1571e-2, 28.90] #thermo polynomial coeff for cp_n2
pn2 = poly1d(n2_pcoef) #Cp_h2o [kJ/kmol-K]
hsn2 = pn2.integ() #h_s function
#
no_pcoef = [-4.187e-9, 0.9747e-5, -0.09395e-2, 29.34] #thermo polynomial coeff for cp_no
pno = poly1d(no_pcoef) #Cp_no [kJ/kmol-K]
hsno = pno.integ() #h_s function
#
co_pcoef = [-2.222e-9, 0.5372e-5, 0.1675e-2, 28.16] #thermo polynomial coeff for cp_co
pco = poly1d(co_pcoef) #Cp_h2o [kJ/kmol-K]
hsco = pco.integ() #h_s function
#
h2_pcoef = [-0.8704e-9, 0.4003e-5, -0.1916e-2, 29.11] #thermo polynomial coeff for cp_h2
ph2 = poly1d(h2_pcoef) #Cp_h2 [kJ/kmol-K]
hsh2 = ph2.integ() #h_s function
#
o2_pcoef = [1.312e-9, -0.7155e-5, 1.520e-2, 25.48] #thermo polynomial coeff for cp_o2
po2 = poly1d(o2_pcoef) #Cp_o2 [kJ/kmol-K]
hso2 = po2.integ() #h_s function

Ok, so now we have all of our nine equations:

**Atom balances:**
1. N-balance
2. C-balance
3. O-balance
4. H-balance

**Kp Equations:**

5. Kp for CO2
6. Kp for H2O
7. Kp for NO

**From the Kp equations we'll also need to sum our moles...**

8. $n_{total} = \sum\limits_{i}n_i$

**And finally our last equation to find the Tad:**

9. 1st Law

In [7]:
#=========================================
# Now we're ready to begin ...

# initial conditions
a_s = 2+6/4
phi = 1.5
aa = a_s/phi
n_c2h6 = 1
n_o2 = aa
n_n2 = aa*3.76
Tin = 298 #K temp of reactants

# Define our equations to solve
def eqn(vars):
    co2, h2o, n2, co, h2, o2, no, nt, Tad  = vars
    eq1 = 2*n_c2h6 - (co2 + co) #carbon balance
    eq2 = 2*n_o2 - (2*co2 + h2o + no + co + 2*o2) #oxygen balance
    eq3 = 6*n_c2h6 - (2*h2o + 2*h2) #hydrogen balance
    eq4 = 2*n_n2 - (2*n2 + no) #nitrogen balance
    eq5 = exp(lnkp_h2o(Tad)) - (h2o / (h2 * o2**0.5))*(1/nt)**(-.5) #Kp_H2O
    eq6 = exp(lnkp_no(Tad)) - (no / (o2**0.5 * n2**0.5)) #Kp_NO
    eq7 = exp(lnkp_co2(Tad)) - (co2 / (co * o2**0.5))*(1/nt)**(-0.5) #Kp_CO2 ... assuming Pref = 1 atm
    eq8 = nt - (co2 + h2o + n2 + co + h2 + no) #total moles
    eq9 = hfc2h6 - (co2*(hfco2+hsco2(Tad)-hsco2(Tin)) + \
                    h2o*(hfh2o+hsh2o(Tad)-hsh2o(Tin)) + \
                    n2*(0.0+hsn2(Tad)-hsn2(Tin)) + \
                    co*(hfco+hsco(Tad)-hsco(Tin)) + \
                    h2*(0.0+hsh2(Tad)-hsh2(Tin)) + \
                    o2*(0.0+hso2(Tad)-hso2(Tin)) + \
                    no*(hfno+hsno(Tad)-hsno(Tin))) #the first law for an adiabatic system, note the use of backslash to separate the lines
    return [eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9]

# Now call the fsolve function to solve our system of equatins
# also note that you'll need a pretty good guess here...
# in this case it helps to realize that the oxygen in the products should be really tiny b/c rich combustion
# alternatively, you could have used a least-squares appraoch to solve
co2, h2o, n2, co, h2, o2, no, nt, Tad = fsolve(eqn, (1, 1, 9, 1, 1, 0.0000001, 0.1, 14, 2000))

# Calculating mole fractions
x_co2 = co2/nt
x_h2o = h2o/nt
x_n2 = n2/nt
x_h2 = h2/nt
x_co = co/nt
ppmv_no = no/nt * 1e6
ppmv_o2 = o2/nt * 1e6

print("The Tad = %1.1f K" % Tad)
print("x_CO2 = %1.4f" % x_co2)
print("x_H2O = %1.4f" % x_h2o)
#print("x_N2 = %1.4f" % x_n2)
#print("x_H2 = %1.4f" % x_h2)
print("ppmv_NO = %1.4f ppmv" % ppmv_no)
#print("ppmv_O2 = %1.f ppmv" % ppmv_o2)

The Tad = 1965.2 K
x_CO2 = 0.0464
x_H2O = 0.1472
ppmv_NO = 6.0612 ppmv


%==================

**Now let's check our answers with cantera:**

In [8]:
# Now we'll create a gas object
# import cantera
gas = ct.Solution('gri30.yaml')

#Let's set what our gas is made of
gas.set_equivalence_ratio(phi, 'C2H6', 'O2:1.0, N2:3.76')
#gas.set_equivalence_ratio(phi=1.5, fuel='C2H6', oxidizer={'O2':1.0, 'N2':3.76}) # alternate form
#gas.X = {'C2H6':1, 'O2':n_o2, 'N2':n_n2} # alternate form ... note slight rounding error in answers

#Now set the initial conditions...
T = None
P = 101325
gas.TP = T, P
print('Let\'s look at the gas properties before we do the Tad calc:')
gas()

#Finally, calculate the adiabatic flame temperature ... remember this is a "HP" type problem.
gas.equilibrate('HP')
print('%=============== ')
print("Tad = %1.1f K" % gas.T)
print('Now, let\'s see the properties after the Tad calc:')
gas() #show the gas properties

# We can list specific elements
print('%=============== Now, picking out the main ones...')
mf_co2 = gas['CO2'].X
print("The mole fraction of CO2 is %1.4f " % mf_co2)
#
mf_h2o = gas['H2O'].X
print("The mole fraction of H2O is %1.4f " % mf_h2o)
#
ppm_no = gas['NO'].X * 1e6
print("The ppmv of NO is %1.4f ppmv " % ppm_no)

Let's look at the gas properties before we do the Tad calc:

  gri30:

       temperature   300 K
          pressure   1.0132e+05 Pa
           density   1.1761 kg/m^3
  mean mol. weight   28.952 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -2.3721e+05       -6.8675e+06  J
   internal energy       -3.2336e+05       -9.3618e+06  J
           entropy            7037.2        2.0374e+05  J/K
    Gibbs function       -2.3484e+06       -6.7989e+07  J
 heat capacity c_p            1073.8             31089  J/K
 heat capacity c_v            786.63             22774  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                O2           0.21301           0.19273            -26.32
              C2H6           0.08579          0.082599           -63.679
             